<a href="https://colab.research.google.com/github/GowthamNats/Recipe-Recommender-System/blob/main/SNA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import os
df = pd.read_json('../content/test.json')
df.ingredients = df.ingredients.apply(lambda ings : [ing.lower() for ing in ings])
df

,id,ingredients
0,18009,"[baking powder, eggs, all-purpose flour, raisi..."
1,28583,"[sugar, egg yolks, corn starch, cream of tarta..."
2,41580,"[sausage links, fennel bulb, fronds, olive oil..."
3,29752,"[meat cuts, file powder, smoked sausage, okra,..."
4,35687,"[ground black pepper, salt, sausage casings, l..."
...,...,...
9939,30246,"[large egg yolks, fresh lemon juice, sugar, bo..."
9940,36028,"[hot sauce, butter, sweet potatoes, adobo sauc..."
9941,22339,"[black pepper, salt, parmigiano reggiano chees..."
9942,42525,"[cheddar cheese, cayenne, paprika, plum tomato..."


In [ ]:
import itertools
from collections import Counter
cooc_counts = Counter()
ing_count  = Counter()
for ingredients in df.ingredients:
    for ing in ingredients:
        ing_count[ing] += 1
    for (ing_a, ing_b) in itertools.combinations(set(ingredients), 2):
       if ing_a > ing_b:
            ing_a, ing_b = ing_b, ing_a
       cooc_counts[(ing_a, ing_b)] += 1

In [ ]:
cooc_df = pd.DataFrame(((ing_a, ing_b, ing_count[ing_a], ing_count[ing_b], cooc) for (ing_a, ing_b), cooc in cooc_counts.items()), columns=['a', 'b', 'a_count', 'b_count', 'cooc'])
cooc_df

,a,b,a_count,b_count,cooc
0,eggs,white sugar,874,299,106
1,milk,white sugar,588,299,61
2,all-purpose flour,white sugar,1184,299,103
3,raisins,white sugar,135,299,23
4,baking powder,white sugar,430,299,63
...,...,...,...,...,...
206259,guacamole,yukon gold potatoes,47,47,1
206260,boiling water,white onion,89,214,1
206261,white onion,yukon gold potatoes,214,47,1
206262,boiling water,yukon gold potatoes,89,47,1


In [ ]:
cooc_df[cooc_df.a == 'chillies'].sort_values('cooc', ascending=False).head(10)

,a,b,a_count,b_count,cooc
5889,chillies,onions,27,2036,15
5837,chillies,salt,27,4485,13
58773,chillies,garlic,27,1791,9
5893,chillies,garam masala,27,254,9
5872,chillies,vegetable oil,27,1131,8
26587,chillies,fresh coriander,27,73,7
104642,chillies,ginger,27,435,7
5858,chillies,tomatoes,27,754,7
5865,chillies,tumeric,27,195,7
5881,chillies,cumin seed,27,254,7


In [ ]:
p_a = cooc_df.a_count / sum(ing_count.values())
p_b = cooc_df.b_count / len(ing_count.values())
p_a_b = cooc_df.cooc / cooc_df.cooc.sum()
cooc_df['pmi'] = np.log(p_a_b / (p_a * p_b))
cooc_df.sort_values('pmi', ascending=False).head(10)

,a,b,a_count,b_count,cooc,pmi
139878,apple brandy,full fat sour cream,1,1,1,6.645209
55421,mccormick black pepper,mccormick perfect pinch italian seasoning,1,1,1,6.645209
150665,mccormick oregano leaves,medium salsa,1,1,1,6.645209
55408,mccormick black pepper,mccormick parsley flakes,1,1,1,6.645209
66302,foie gras,medjool date,1,1,1,6.645209
160135,disco empanada frozen,knorr® beef flavored bouillon cube,1,1,1,6.645209
150666,mccormick ground cumin,medium salsa,1,1,1,6.645209
186756,chocolate candy bars,semisweet chocolate chunks,1,1,1,6.645209
133717,foster farms chicken drumsticks,thai chili garlic sauce,1,1,1,6.645209
133709,caramel ice cream topping,thai chili garlic sauce,1,1,1,6.645209


In [ ]:
min_count = 5
cooc_df[(cooc_df.a_count >= min_count) & (cooc_df.b_count >= min_count)].sort_values('pmi', ascending=False).head(20)

,a,b,a_count,b_count,cooc,pmi
160791,açai,granola,5,5,3,4.524945
48357,frozen whipped topping,yellow cake mix,5,5,2,4.119480
101600,chia seeds,granola,5,5,2,4.119480
160798,açai,goji berries,5,5,2,4.119480
680,glace cherries,sultana,5,8,3,4.054941
43696,pepperoncini,roast beef,5,6,2,3.937158
647,glace cherries,mixed spice,5,9,3,3.937158
48355,frozen whipped topping,vanilla instant pudding,5,6,2,3.937158
76373,asafoetida powder,split black lentils,12,7,5,3.823830
155866,boneless sirloin steak,less sodium beef broth,6,6,2,3.754837


In [ ]:
min_count = 30
cooc_df[(cooc_df.a_count >= min_count) & (cooc_df.b_count >= min_count) & (cooc_df.cooc > 1)].sort_values('pmi', ascending=True).head(20)

,a,b,a_count,b_count,cooc,pmi
42112,garlic cloves,white sugar,1535,299,2,-5.698374
5289,fine sea salt,salt,78,4485,2,-5.426847
86070,grated parmesan cheese,ground cumin,481,684,2,-5.365469
20493,garlic cloves,minced garlic,1535,418,4,-5.340264
54976,oil,unsalted butter,456,692,2,-5.323723
41543,sesame oil,tomatoes,410,754,2,-5.303194
41700,cooking spray,soy sauce,364,824,2,-5.272969
30027,salsa,sugar,258,1630,3,-5.205474
94326,all-purpose flour,fresh lime juice,1184,346,3,-5.179272
34456,garlic,large garlic cloves,1791,223,3,-5.153880


In [ ]:
from scipy.sparse import csr_matrix
data_df = cooc_df[cooc_df.pmi > 0].copy()
# Since the matrix is symetric, we add the same values for (b,a) as we have for (a,b)
data_df_t = data_df.copy()
data_df.a, data_df.b = data_df.b, data_df.a
data_df = pd.concat([data_df, data_df_t])

rows_idx, row_keys = pd.factorize(data_df.a)
cols_idx, col_keys = pd.factorize(data_df.b)
values = data_df.pmi

matrix = csr_matrix((values, (rows_idx, cols_idx)))
key_to_row = {key: idx for idx, key in enumerate(row_keys)}
matrix

<4462x4150 sparse matrix of type '<class 'numpy.float64'>'
	with 63891 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(200)
factors = svd.fit_transform(matrix)
factors

array([[ 2.07776241e-03,  3.19647611e-02,  2.53713481e-03, ...,
        -5.77090326e-02, -3.35319942e-02,  1.30960712e-01],
       [ 1.75389993e-02,  9.66702171e-01, -5.67548195e-03, ...,
        -6.17001436e-01, -9.63861404e-02,  2.77157220e+00],
       [ 6.87931190e-03,  2.26305156e-01,  1.49949095e-02, ...,
        -7.94222058e-02, -1.77904707e-01,  2.99733062e-01],
       ...,
       [ 3.77205744e-03,  9.79107284e-03,  2.41067822e-03, ...,
         1.44772173e-01,  6.22055043e-02, -8.15328068e-02],
       [ 2.06462807e-03,  2.53458115e-03,  3.43112904e-02, ...,
         2.99647153e-02, -2.75088927e-02,  4.99234626e-02],
       [ 8.71447148e-04,  3.89409281e-04,  8.31346706e-04, ...,
        -4.42626821e-01,  5.98395917e-01, -2.61321234e-02]])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
def most_similar(ingredient, topn=10):
    if ingredient not in key_to_row:
        print("Unknown ingredient.")
    factor = factors[key_to_row[ingredient]]
    cosines = cosine_similarity([factor], factors)[0]
    indices = cosines.argsort()[::-1][:topn + 1]
    keys = [row_keys[idx] for idx in indices if idx != key_to_row[ingredient]]
    return keys, cosines[indices]

def display_most_similar(ingredient, topn=10):
    print("- Most similar to '{}'".format(ingredient))
    for similar_ing, score in zip(*most_similar(ingredient, topn)):
        print("  . {} : {:.2f}".format(similar_ing, score)) 

In [ ]:
display_most_similar('chile powder')

- Most similar to 'chile powder'
  . fully cooked luncheon meat : 1.00
  . sliced cucumber : 0.77
  . kim chee : 0.71
  . marjoram : 0.71
  . sliced turkey : 0.70
  . greek seasoning : 0.70
  . baby portobello mushrooms : 0.69
  . roast red peppers, drain : 0.67
  . feta cheese : 0.67
  . lite coconut milk : 0.66


In [ ]:
import random

def recommendations(arr):
  final = []
  temp = arr
  final.append(get_recipes(temp))
  for i in range(0, len(arr)):
    initial = temp[i]
    temp2 = most_similar(arr[i])[0]
    for j in temp2:
      temp[i] = j
      final.append(get_recipes(temp))
    temp[i] = initial
  print("Recommended recipes are as follows:")
  for k in (final[0]):
    print(k)

def get_recipes(arr):
  final = []
  for ingredients in df.ingredients:
    flag = 0
    for i in arr:
      if i not in ingredients:
        flag = 1
    if flag == 0:
      final.append(ingredients)
  return final

recommendations(["tomatoes", "thyme"])

Recommended recipes are as follows:
['red kidney beans', 'sugar', 'coriander seeds', 'garlic', 'corn starch', 'red chili powder', 'water', 'jalapeno chilies', 'salt', 'ground oregano', 'tomatoes', 'white onion', 'chuck roast', 'purple onion', 'chipotle chile powder', 'cheddar cheese', 'lime juice', 'bacon', 'thyme', 'ground cumin']
['water', 'salt', 'onions', 'tomatoes', 'grated parmesan cheese', 'oil', 'oregano', 'eggs', 'garlic', 'thyme', 'pepper flakes', 'eggplant', 'ricotta', 'fresh basil leaves']
['white onion', 'parsley', 'salt', 'sun-dried tomatoes in oil', 'tomato paste', 'bay leaves', 'basil', 'green pepper', 'oregano', 'tomatoes', 'green onions', 'garlic', 'thyme', 'celery ribs', 'water', 'red wine', 'yellow onion', 'yellow peppers']
['homemade chicken stock', 'meat', 'blanched almonds', 'lard', 'oregano', 'clove', 'guajillo chiles', 'tomatillos', 'allspice berries', 'ancho chile pepper', 'brioche', 'sesame seeds', 'grated nutmeg', 'thyme', 'onions', 'tomatoes', 'fresh ginger